In [6]:
from pyfuseki import FusekiUpdate
import httpx
from rdflib import Graph

In [2]:
upAcervo = FusekiUpdate('http://localhost:3030', 'acervo')

DELETE GRAPHS

In [3]:
d = """DELETE { graph ?g { ?s ?p ?o } } 
        WHERE {
        graph ?g {?s ?p ?o.}
        }"""

response = upAcervo.run_sparql(d)
response

In [4]:
response.convert()

{'statusCode': 200, 'message': 'Update succeeded'}

WORK BK

In [10]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/bibframe/work/bk-23-1"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('work.nt')

<Graph identifier=N10ed1f546471498b81448e9ce2b09c24 (<class 'rdflib.graph.Graph'>)>

INSTANCE BK

In [11]:
uri = "http://localhost:3030/acervo?graph=https://bibliokeia.com/bibframe/instance/bk-23-1"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('instance.nt')

<Graph identifier=N1b52b00a4adb49be804fb4ac1eed61ac (<class 'rdflib.graph.Graph'>)>

ITEMS BK

In [15]:
uri = 'http://id.loc.gov/resources/items/2215252-050-9.rdf'
g = Graph()
g.parse(uri)
g.serialize('item.nt')

<Graph identifier=N81349e2591274bada155170aeee81ea7 (<class 'rdflib.graph.Graph'>)>

NAMES BK

In [14]:
uri = "http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/names/n79029745"
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('name.nt')

<Graph identifier=Nef4313330ebc4defaa4fab3a9618e9fe (<class 'rdflib.graph.Graph'>)>

SUBJECTS BK

In [8]:
token = "sh85084414"
uri = f'http://localhost:3030/thesaurus?graph=https://bibliokeia.com/authorities/subjects/{token}'
r = httpx.get(uri)
g = Graph()
g.parse(r.content)
g.serialize('subjects.nt')

<Graph identifier=N8687b45b63cc46f8bed88bf7903542c0 (<class 'rdflib.graph.Graph'>)>